<a href="https://colab.research.google.com/github/nikomen/CBE_AI/blob/main/AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Podłączenie google drive z Colabem
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [84]:
import pandas as pd
import numpy as np

#Wczytanie danych
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/all.csv')

In [ ]:
# Miałem problem z IP oraz czasem jako danymi. Na razie je wywaliłem
dataset = dataset.drop(['SourceIP','DestinationIP','PacketTimeMode','TimeStamp'],1)
# Wywalam duplikaty i próbki które nie miały wszystch cech
dataset = dataset.dropna()
dataset = dataset.drop_duplicates()
dataset

In [86]:
# Sprawdzenie jak zbalansowane są dane
dataset.groupby(dataset.DoH).size()

DoH
False    353122
True      16039
dtype: int64

In [87]:
# Balansowanie danych
from sklearn.utils import resample
Klasa_mniejsza = dataset[dataset.DoH == True]
Klasa_wieksza = dataset[dataset.DoH == False]
Klasa_wieksza_zmniejszona = resample(Klasa_wieksza, replace=False, n_samples=16039, random_state=42)
dataset = pd.concat([Klasa_wieksza_zmniejszona, Klasa_mniejsza])
dataset.groupby(dataset.DoH).size()

DoH
False    16039
True     16039
dtype: int64

In [88]:
# Dzielę dane na cechy i etykiety
X = dataset.iloc[:, :-1]
# Zamiana wartości etykiet z True/False na 1/0.
y = dataset.iloc[:, -1].astype(int)

# Standaryzacja danych Standaryzacja polega na przekształceniu danych w taki sposób, że średnia wartość (średnia arytmetyczna) wszystkich danych wynosi 0, a odchylenie standardowe wynosi 1.
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler()
X = X_scaled.fit_transform(X)

In [89]:
# Podział na dane treningowe i dane testowe
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [90]:
# Wybór X najlepszych cech
from sklearn.feature_selection import SelectKBest, mutual_info_classif
select_feature = SelectKBest(mutual_info_classif, k=17).fit(X_train,y_train)
X_train = select_feature.transform(X_train)
X_test = select_feature.transform(X_test)

In [91]:
# Wybieram algorytm uczenia maszynowego
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=1442)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [93]:
# Wybieram algorytm uczenia maszynowego
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [95]:
# Wybieram algorytm uczenia maszynowego - Random Forest (RF),
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [97]:
# Wybieram algorytm uczenia maszynowego - support Vector Machines
from sklearn import svm
model = svm.SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [99]:
# Wybieram algorytm uczenia maszynowego - deep neural network DNN
from sklearn.neural_network import MLPClassifier
MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [100]:
# Sprawdzam jak model sobie poradził
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Accuracy - Miara dokładności klasyfikacji.
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Precision - Miara dokładności pozytywnych predykcji (tp / (tp + fp)).
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Recall - Miara pokrycia pozytywnych przypadków (tp / (tp + fn)).
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# F1 Score - Średnia harmoniczna precision i recall.
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

# Confusion Matrix - Macierz błędów, która zawiera informacje o tp, tn, fp i fn.
confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion)


Accuracy: 0.8768079800498753
Precision: 0.8503079109426812
Recall: 0.9097820577800304
F1 Score: 0.8790401567091087
Confusion Matrix:
[[3442  632]
 [ 356 3590]]


In [101]:
#Cross-validation score (wynik walidacji krzyżowej) to metoda oceny wydajności modelu uczenia maszynowego, która pomaga w określeniu, jak dobrze model generalizuje na nie widziane wcześniej dane
cv_score = cross_val_score(model, X,y,cv=5,scoring="accuracy")
cv_score.mean()

0.8958756849108717